In [ ]:
pip install riotwatcher

In [ ]:
from riotwatcher import LolWatcher
import pandas as pd
lol_watcher = LolWatcher('your api key here')

In [321]:
#la variable mi region guardara el valor necesario para identificar la plataforma de la cual queremos obtener la informacion
myRegion = 'la1'
#seteamos el valor de nombre de invocador en la variable mySummonerName
mySummonerName='SquareHammer'
#la variable me alojara los valores de el perfil de invocador
me = lol_watcher.summoner.by_name(myRegion, mySummonerName)
#mostramos el contenido de la variable me
print(me)
#obtenemos los datos de clasificatoria del nombre de invocador proporcionado.
myprofile = lol_watcher.league.by_summoner(myRegion, me['id'])
#mostramos el contenido de la variable myprofile
print(myprofile)



{'id': 'VD_g_pCYuvPPFOV_waLjp0JBg0dsfipWKKS_7qSJ-M8C', 'accountId': 'vRgfRW5Ebfb4_VMXND8ECORbqXyqcQgkNSoP1NW92vk', 'puuid': 'bKIY8nxkX7Dt4M8GMtOMRHgfmQ71AvJBqPTzrIvLHcdm2jGacY4OoJHEZ5alUS3TAFmch8UZzLQW9w', 'name': 'SquareHammer', 'profileIconId': 4887, 'revisionDate': 1631748868000, 'summonerLevel': 597}
[{'leagueId': 'c0e7ace6-3085-4e33-91b3-f06866df90da', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'SILVER', 'rank': 'II', 'summonerId': 'VD_g_pCYuvPPFOV_waLjp0JBg0dsfipWKKS_7qSJ-M8C', 'summonerName': 'SquareHammer', 'leaguePoints': 58, 'wins': 139, 'losses': 155, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


In [322]:
import json
#De lol_watcher pordemos obtener la lista de partidas dada una cuenta proporcionada 
#Le pasamos los parametros de region y accountId
my_matches = lol_watcher.match.matchlist_by_account(myRegion, me['accountId'])

# Una vez obtenidos los datos de my_matches los normalizamos y los asignamos aun dataframe:
dftest=pd.DataFrame
dftest=pd.json_normalize(my_matches,record_path=['matches'])
dfRanked=pd.DataFrame
dfRanked=dftest

#filtramos los valores por cola 420 se refiere a la cola clasificatoria en solitario y posicion al carril que se defiende dentro de la grieta del invocador.
#que son 5: Bottom, Support, Mid, Jungle y Top, en mi caso normalmente participo en la posicion de top.
dfRanked=dftest.query("queue==420 and lane=='TOP'")

#convertimos la fercha de formato POSIX a Formato Humano
dfRanked['timestamp']= pd.to_datetime(dfRanked['timestamp'],unit='ms')

#ordenamos el dataframe por fecha
dfRanked.sort_values(by=['timestamp'])
#mostramos el dataframe 
display(dfRanked)

# Primero obtenemos la version mas reciente de los campeones de data dragon
versions = lol_watcher.data_dragon.versions_for_region('na1')['n']['champion']

#Una vez obtenida la version extraemos la lista de campeones con su respectivo id  y nombre 
listaDeCampeones = lol_watcher.data_dragon.champions(versions, False, 'en_US')

#iteramos la lista de campeones 
champ_dict = []
for key in listaDeCampeones['data']:
    champcolection={}
    row = listaDeCampeones['data'][key]
    champcolection["champion"] = int(row['key'])
    champcolection["ChampionName"] = row['name']
    champ_dict.append(champcolection)
    dfCampeones = pd.DataFrame(champ_dict)


display(dfCampeones)



#Hacemos un request para obtener la informacion en formato json de los tipos de colas.
response=urlopen("https://static.developer.riotgames.com/docs/lol/queues.json")
#Asignamos la informacion a la variable dfColas
dfColas=pd.read_json(response)
display(dfColas)

#Hago una combinacion de dataframes para obtener los nombres de los campeones
dfRanked=pd.merge(dfRanked,dfCampeones)
#Renombro la columna queue a queueId porque en el dataframe del historial de partidas no coincide el nombre con el de las colas.
dfRanked.rename(columns={'queue':'queueId'}, inplace=True)
#Combinamos el dataframes de las colas y el del historial de partidas
dfRanked=pd.merge(dfRanked,dfColas)
#mostramos las columnas del dataframe combinado
dfRanked.columns
#Mostramos el resultado final de la combinacion de 3 dataframes:
dfRanked[['platformId', 'gameId', 'ChampionName' ,'description', 'season', 'timestamp','role', 'lane']]





<ipython-input-322-5c1832fb4504>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfRanked['timestamp']= pd.to_datetime(dfRanked['timestamp'],unit='ms')


,platformId,gameId,champion,queue,season,timestamp,role,lane
18,LA1,1134480163,10,420,13,2021-09-08 03:35:36.982,SOLO,TOP
37,LA1,1132956665,131,420,13,2021-09-05 01:46:32.036,SOLO,TOP
41,LA1,1131734199,131,420,13,2021-09-02 06:47:43.646,SOLO,TOP
42,LA1,1131679021,41,420,13,2021-09-02 03:58:23.588,SOLO,TOP
45,LA1,1131207931,41,420,13,2021-09-01 03:18:23.078,SOLO,TOP
48,LA1,1130221810,41,420,13,2021-08-30 02:00:50.229,SOLO,TOP
49,LA1,1129928568,41,420,13,2021-08-29 17:42:23.522,SOLO,TOP
50,LA1,1128838347,875,420,13,2021-08-27 04:47:21.974,SOLO,TOP
51,LA1,1128302345,17,420,13,2021-08-26 03:01:19.124,SOLO,TOP
52,LA1,1126924803,41,420,13,2021-08-23 06:05:54.762,SOLO,TOP


,champion,ChampionName
0,266,Aatrox
1,103,Ahri
2,84,Akali
3,166,Akshan
4,12,Alistar
...,...,...
151,238,Zed
152,115,Ziggs
153,26,Zilean
154,142,Zoe


,queueId,map,description,notes
0,0,Custom games,None,None
1,2,Summoner's Rift,5v5 Blind Pick games,Deprecated in patch 7.19 in favor of queueId 430
2,4,Summoner's Rift,5v5 Ranked Solo games,Deprecated in favor of queueId 420
3,6,Summoner's Rift,5v5 Ranked Premade games,Game mode deprecated
4,7,Summoner's Rift,Co-op vs AI games,Deprecated in favor of queueId 32 and 33
...,...,...,...,...
78,1300,Nexus Blitz,Nexus Blitz games,None
79,1400,Summoner's Rift,Ultimate Spellbook games,None
80,2000,Summoner's Rift,Tutorial 1,None
81,2010,Summoner's Rift,Tutorial 2,None


,platformId,gameId,ChampionName,description,season,timestamp,role,lane
0,LA1,1134480163,Kayle,5v5 Ranked Solo games,13,2021-09-08 03:35:36.982,SOLO,TOP
1,LA1,1132956665,Diana,5v5 Ranked Solo games,13,2021-09-05 01:46:32.036,SOLO,TOP
2,LA1,1131734199,Diana,5v5 Ranked Solo games,13,2021-09-02 06:47:43.646,SOLO,TOP
3,LA1,1131679021,Gangplank,5v5 Ranked Solo games,13,2021-09-02 03:58:23.588,SOLO,TOP
4,LA1,1131207931,Gangplank,5v5 Ranked Solo games,13,2021-09-01 03:18:23.078,SOLO,TOP
5,LA1,1130221810,Gangplank,5v5 Ranked Solo games,13,2021-08-30 02:00:50.229,SOLO,TOP
6,LA1,1129928568,Gangplank,5v5 Ranked Solo games,13,2021-08-29 17:42:23.522,SOLO,TOP
7,LA1,1126924803,Gangplank,5v5 Ranked Solo games,13,2021-08-23 06:05:54.762,SOLO,TOP
8,LA1,1118734509,Gangplank,5v5 Ranked Solo games,13,2021-08-07 02:40:10.962,SOLO,TOP
9,LA1,1109650423,Gangplank,5v5 Ranked Solo games,13,2021-07-20 06:00:31.988,SOLO,TOP


In [325]:
#obtenemos la lista de partidad por cueenta de usuario
my_matches = lol_watcher.match.matchlist_by_account(myRegion, me['accountId'])
#Para fines visuales solo obtendremos la ultima partida jugada
last_match = my_matches['matches'][0]
#obtenemos los datos detallados de la ultima partida
match_detail = lol_watcher.match.by_id(myRegion, last_match['gameId'])
#creamos una lista para los participantes
participantes = []
#iteramos de acuerdo a cada participante
for row in match_detail['participants']:
    participantes_row = {}
    participantes_row['champion'] = row['championId']#se estrae el identificador del campeon
    participantes_row['spell1'] = row['spell1Id']#se extrae el identificador del primer hechizo de invocador
    participantes_row['spell2'] = row['spell2Id']#se extrae el identificador del segundo hechizo de invocador
    participantes_row['win'] = row['stats']['win']#Extraemos el valor booleano que nos indica si fue una victoria o una derrota
    participantes_row['kills'] = row['stats']['kills']#obtenemos la cantidad de asesinatos que se lograron en el juego
    participantes_row['deaths'] = row['stats']['deaths']#obtenemos la cantidad de muertes que se lograron en el juego
    participantes_row['assists'] = row['stats']['assists']#obtenemos la cantidad de assitencias que se lograron en el juego
    participantes_row['totalDamageDealt'] = row['stats']['totalDamageDealt']#obtenemos la cantidad de total de daño infligido en la partida
    participantes_row['goldEarned'] = row['stats']['goldEarned']#obtenemos la cantidad de oro que consiguio en la partida
    participantes_row['champLevel'] = row['stats']['champLevel']#obtenemos el nivel maximo alcanzado por el campeon 
    participantes_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']#obtenemos la cantidad de minions asesinados durante la partida
    participantes_row['item0'] = row['stats']['item0']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes_row['item1'] = row['stats']['item1']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes_row['item2'] = row['stats']['item2']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes_row['item3'] = row['stats']['item3']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes_row['item4'] = row['stats']['item4']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes_row['item5'] = row['stats']['item5']#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
    participantes.append(participantes_row)
df = pd.DataFrame(participantes)
df

,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,item2,item3,item4,item5
0,131,32,4,True,9,9,16,51203,11599,16,34,4636,3157,3158,4629,0,0
1,133,4,3,True,3,9,17,38140,10102,15,37,6673,3006,3095,1038,0,0
2,80,14,4,True,10,7,19,49214,12605,15,38,6691,3814,2031,3111,6676,1036
3,8,7,4,True,6,8,18,37970,10470,15,33,4636,3020,4629,3916,1026,1028
4,91,4,14,True,14,9,6,46118,11259,15,25,6691,3142,6676,3158,1036,0
5,236,4,3,False,16,9,14,46231,12078,14,37,3508,6671,3006,3123,1042,3072
6,83,4,32,False,3,8,22,32984,9460,14,25,6632,3111,3053,3211,1028,0
7,777,4,14,False,1,12,11,19518,8153,13,18,6673,3111,3091,2031,0,0
8,110,4,7,False,10,8,23,42802,11308,15,48,3142,6691,3814,3158,3133,0
9,161,4,13,False,12,5,19,57569,11980,16,40,6653,3020,3916,4628,3089,0


In [ ]:
def SummonerName(match_summoners,participant):
      #Aqui recibo los participantes de la partida
      participantIdentities=match_summoners
      Names=[]
      for y in participantIdentities['participantIdentities']:
            #comparo que el participantId sea el que corresponde y regreso el nombre de invocador
            if participant ==  y['participantId']:
                       return  y['player']['summonerName']
            else:
                  continue
      return 'Nadie'




In [ ]:
def ChampionName(champion):
   
      for index, z in dfCampeones.iterrows():
             #si el campeon que recibo lo encuetro en mi dataframe devuelvo el nombre del campeon
            if int(champion) ==  int(z['champion']):
                 return  z['ChampionName']
            else:continue

      return 'Nadie'

In [327]:
#Obtenemos los hechizos de invocador de data dragon
spells=lol_watcher.data_dragon.summoner_spells(versions)
#creamos una lista de hechizos de invocador
hechizosInvocador=[]
#creamos el dataframe 
dfHechizos=pd.DataFrame
#iteramos 
for key in spells['data']:
    aux={}
    #obtenemos los datos individuales de cada hechizo de invocador
    row = spells['data'][key]
    #normalizamos el archivo json y lo asignamos a un dataframe
    dfHechizos=pd.json_normalize(row)    
    #Obtenemos los valores que no interesan, en este caso nombre e identificador de los hechizos de invocador
    #como solo estamos obteniendo una sola fila por iteracion sabemos que obtenemos el valor ubicado en la fila 0
    aux['Name']=dfHechizos['name'].values[0]
    aux['Spell']=dfHechizos['key'].values[0]
    #agregamos la informacion a la lista.
    hechizosInvocador.append(aux)
#asignamos a un dataframe la informacion obtenida 
dfSpell=pd.DataFrame(hechizosInvocador)    
#mostramos el dataframe
dfSpell
          


,Name,Spell
0,Barrier,21
1,Cleanse,1
2,Ignite,14
3,Exhaust,3
4,Flash,4
5,Ghost,6
6,Heal,7
7,Clarity,13
8,To the King!,30
9,Poro Toss,31


In [332]:
#creamos una lista de participantes
participantes = []
#iteramos el dataframe con la informacion del historial de partidas
for index,x in dfRanked.iterrows():
      #Extraemos la informacion de las partidas utilizando los campos region=la1  y gameId=lo obtenemos del histortial de partidas
      detalleDePartidas = lol_watcher.match.by_id(myRegion, int(x['gameId']))
      #iteramos cada partida y extraemos datos relevantes de ella
      for row in detalleDePartidas['participants']:
        participantes_row = {}
        participantes_row['gameId'] = x['gameId'] #Guardamos el gameId de la partida
        participantes_row['Fecha'] = x['timestamp']#Guardamos la fecha
        participantes_row['participantId'] = row['stats']['participantId']#ParticipantId representa a cada jugador con un numero del 1 al 10 dentro de la misma
        #Como en el json de la informacion el participantId se repite constantemente y el nombre del invocador no estaba en el mismo nivel en el json tenia que ir
        #a buscarlo un nivel mas adentro que estaba fuera del alcance de la informacion que estamos iterando por lo que tuce que crear una funcion que me regresara
        #el valor que necesitaba al mismo tiempo que iba poblando el dataframe ya que una vez que lo poblara sin hacer este paso, seria muy dificil hacer.
        participantes_row['SummonerName'] = SummonerName(detalleDePartidas,row['stats']['participantId'])
        participantes_row['champion'] = ChampionName(row['championId'])#obtenemos el nombre del campeon que utilizo cada jugador enla partida.
        participantes_row['spell1'] = spellName(row['spell1Id'])#obtenemos el el hechizo de invocador 
        participantes_row['spell2'] = spellName(row['spell2Id'])#obtenemos el el hechizo de invocador 
        participantes_row['win'] = row['stats']['win']#Extraemos el valor booleano que nos indica si fue una victoria o una derrota
        participantes_row['kills'] = row['stats']['kills']#extraemos los asesinatos durante la partdia
        participantes_row['deaths'] = row['stats']['deaths']#obtenemos la cantidad de muertes que se lograron en el juego
        participantes_row['assists'] = row['stats']['assists']#obtenemos la cantidad de assitencias que se lograron en el juego
        participantes_row['totalDamageDealt'] = row['stats']['totalDamageDealt']#obtenemos la cantidad de total de daño infligido en la partida
        participantes_row['goldEarned'] = row['stats']['goldEarned']#obtenemos la cantidad de oro que consiguio en la partida
        participantes_row['champLevel'] = row['stats']['champLevel']#obtenemos el nivel maximo alcanzado por el campeon 
        participantes_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']#obtenemos la cantidad de minions asesinados durante la partida
        participantes_row['item0'] = ItemsName(row['stats']['item0'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes_row['item1'] = ItemsName(row['stats']['item1'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes_row['item2'] = ItemsName(row['stats']['item2'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes_row['item3'] = ItemsName(row['stats']['item3'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes_row['item4'] = ItemsName(row['stats']['item4'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes_row['item5'] = ItemsName(row['stats']['item5'])#Obtenemos el identificador de 1 de los 6 item maximos que se pueden comprar en la partida
        participantes.append(participantes_row)
        dfPartidasJugadas = pd.DataFrame(participantes)
#limpiamos participantes porque si no se limpiaba siempre se devolvia el primero de la lista en todos los resultado
participantes.clear();
#Ordenamos por gameid y participantId
dfPartidasJugadas=dfPartidasJugadas.sort_values(by=['gameId', 'participantId'])
#Mostramos el dataframe
display(dfPartidasJugadas)

,gameId,Fecha,participantId,SummonerName,champion,spell1,spell2,win,kills,deaths,...,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,item2,item3,item4,item5
180,1103130247,2021-07-07 06:25:57.724,1,MichiDeprimido,Mordekaiser,Teleport,Ignite,False,2,4,...,99008,8307,13,139,Escudo de Doran,Abrazo Demoniaco,Punteras de Acero Revestidas,Égida de Fuego Solar,NONE,NONE
181,1103130247,2021-07-07 06:25:57.724,2,PowersZ,Rek'Sai,Smite,Flash,False,3,8,...,138473,8979,13,7,Filo de la Noche,Garra del Merodeador,Vesta Espinosa,NONE,NONE,Punteras de Acero Revestidas
182,1103130247,2021-07-07 06:25:57.724,3,SquareHammer,Teemo,Ignite,Flash,False,1,7,...,150216,9871,15,209,Botas del Hechicero,Diente de Nashor,Angustia de Liandry,Orbe del Olvido,Cinturón de Gigante,NONE
183,1103130247,2021-07-07 06:25:57.724,4,Campe0n Rushchan,Xerath,Flash,Barrier,False,3,1,...,76651,9051,12,66,Tempestad de Luden,Enfoque al Horizonte,Fragmento de Hielo Puro,Botas del Hechicero,Lágrima de la Diosa,NONE
184,1103130247,2021-07-07 06:25:57.724,5,Save My Grave,Caitlyn,Heal,Flash,False,8,2,...,168933,12406,14,213,Cañón de Fuego Rápido,Filo de la Tormenta,Último Suspiro,Grebas del Berserker,Matakrakens,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1134480163,2021-09-08 03:35:36.982,6,Sandpie7,Graves,Flash,Smite,False,5,5,...,180304,11780,14,30,Eclipse,Punteras de Acero Revestidas,Cuchilla Oscura,El Coleccionista,Centinela de Control,NONE
6,1134480163,2021-09-08 03:35:36.982,7,Johnstep,Veigar,Ignite,Flash,False,9,6,...,143399,12621,16,162,Impulso Cósmico,Reloj de Arena de Zhonya,Hielo Eterno,Botas de Mercurio,Orbe del Olvido,Varita Explosiva
7,1134480163,2021-09-08 03:35:36.982,8,SquareHammer,Kayle,Teleport,Flash,False,2,9,...,151799,11199,15,187,Al Filo de la Cordura,NONE,Diente de Nashor,Grebas del Berserker,Agrietador,NONE
8,1134480163,2021-09-08 03:35:36.982,9,ToxicRaiven0w0,Jhin,Exhaust,Flash,False,4,4,...,123741,10167,14,136,Espada de Doran,Fuerza del Viento,Recuerdos de Lord Dominik,Botas de Rapidez,Puñal Serrado,Capa de Agilidad


In [329]:
#AL igual que los hechizos de invocador, obtenemos los objetos de data dragon
items=lol_watcher.data_dragon.items(versions,locale='es_MX')
#Creamos una lista 
Objetos=[]    
for key in items['data']:
        itemExtraction={}
        #key nos guarda el identificador del objeto y con ese identificador nos adentramos a toda la información 
        #del objeto
        row = items['data'][key]
        #Guardamos la informacion que necesitamos: Nombre y item0 que en este caso es el identificador
        itemExtraction['Name'] = row['name']
        itemExtraction['item0'] = int(key)
        #agregamos la información a la lista
        Objetos.append(itemExtraction)
#creamos el datadrame con la información de la lista
df6 = pd.DataFrame(Objetos)
#Agremos el identificador 0  con descripción 'none' en las partidas no siempre se llega al total de numero de objetos posibles
#por lo que en varias nos encontramos con el valor 0 y no habia algo que relacionara esa información con los objetos.
df6=df6.append({'Name': 'NONE','item0' : 0},ignore_index=True)
#mostramos el dataframe
display(df6)

,Name,item0
0,Botas,1001
1,Amuleto de las Hadas,1004
2,Perla de Rejuvenecimiento,1006
3,Cinturón de Gigante,1011
4,Capa de Agilidad,1018
...,...,...
195,Garra del Merodeador,6693
196,Rencor de Serylda,6694
197,Colmillo de Serpiente,6695
198,Cadenas de Anatema,8001


In [ ]:
def ItemsName(itemid):
       #iteramos el dataframe con los objetos
      for index, z in df6.iterrows():
            #si el objeto que recibo lo encuetro en mi dataframe devuelvo el nombre del objeto
            if int(itemid) ==  int(z['item0']):
                 return  z['Name']
            else:continue

      return 'None'

In [ ]:
def spellName(spellId):
       #iteramos el dataframe con los hechizos de invocador
      for index, z in dfSpell.iterrows():
            #si el hechizo que recibo lo encuetro en mi dataframe devuelvo el nombre del hechizo de invocador
            if int(spellId) ==  int(z['Spell']):
                 return  z['Name']
            else:continue

      return 'None'